In [46]:
import psycopg2
import pandas as pd
from IPython.display import display, Markdown
from plotly_calplot import calplot

pd.set_option('display.max_columns', None)

In [2]:
# Set up the connection parameters
server_address = '10.0.0.1'
port = 5432
db_name = 'dsn'
db_username = 'udabb'
db_password = '43bc7d679f'

In [5]:
mapping = {
#     "01": "Poste éligible à l'aide en entreprise adaptée",
    "02": "Poste éligible à l'aide en structure d’IAE : ACI (ACI_DC)",
    "03": "Poste éligible à l'aide en structure d’IAE : AI (AI_DC)",
    "04": "Poste éligible à l'aide en structure d’IAE : EI (EI_DC)",
    "05": "Poste éligible à l'aide en structure d’IAE : ETTI (ETTI_DC)",
#     "06": "Poste éligible à l'aide en structure d’IAE : ACI en milieu pénitentiaire (ACI_MP)",
#     "07": "Poste éligible à l'aide en structure d’IAE : EI en milieu pénitentiaire (EI_MP)",
}

In [6]:
def df_from_query(query):
    conn_string = f"host={server_address} port={port} dbname={db_name} user={db_username} password={db_password}"
    connection = psycopg2.connect(conn_string)
    cursor = connection.cursor()
    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    cursor.close()
    connection.close()
    return pd.DataFrame([r for r in results], columns=column_names)

def list_to_enum(my_list):
    return "'" + "', '".join(map(str, my_list)) + "'"

In [45]:
# récupération des id_empass des contrats IAE
id_employeur_assure = list(df_from_query(
    "SELECT DISTINCT id_employeur_assure FROM public.ddadtcontrat WHERE "
    f"complement_dispositif_public IN ({list_to_enum(mapping.keys())}) "
#     "AND unite_mesure = '10' "
#     "LIMIT 50"
    ";"
)['id_employeur_assure'])
len(id_employeur_assure)

40607

In [19]:
# récupération des versements liés aux id_empass
versement = df_from_query(
    "SELECT DISTINCT * FROM dadeh.ddadtversement WHERE "
    f"id_employeur_assure IN ({list_to_enum(id_employeur_assure)}) "
    ";"
)
id_versement = list(versement['id'].drop_duplicates())
# versement

In [20]:
# récupération des rembrutes liés aux id_versements
rembrute = df_from_query(
    "SELECT DISTINCT * FROM dadeh.ddadtrem_brute_ac WHERE "
    f"id_versement IN ({list_to_enum(id_versement)}) "
    ";"
)
id_rembrute = list(rembrute['id'].drop_duplicates())
# rembrute

In [21]:
def harmonize_unite(row):
    if row['unite_mesure'] == '10':
        return row
    # tout le reste on considère que c'est en jours
    row['mesure'] *= 4.8
    row['unite_mesure'] = '10r'
    return row

# récupération des activites liés aux id_rembrutes
activite = df_from_query(
    "SELECT DISTINCT * FROM dadeh.ddadtactivite WHERE "
    f"id_remuneration_brute_ac IN ({list_to_enum(id_rembrute)}) "
    ";"
)
# drop les NA et 40 car cas de subrogation de la sécurité sociale
activite_processed = activite.loc[~(activite['unite_mesure']=='40')].dropna(subset='unite_mesure')
activite_processed = activite_processed.apply(lambda df: harmonize_unite(df), axis=1)
# activite_processed
# activite['unite_mesure'].value_counts(dropna=False)

In [12]:
# merge avec les rembrutes pour avoir les dates de paies et id_versement
merged = pd.merge(
    activite_processed[['id_remuneration_brute_ac', 'type', 'mesure', 'unite_mesure']],
    rembrute,
    left_on='id_remuneration_brute_ac',
    right_on='id',
    how='left',
)
merged

,id_remuneration_brute_ac,type,mesure,unite_mesure,id,id_versement,numero_contrat,date_debut_paie,date_fin_paie,montant
0,1676863629,02,0.00,10,1676863629,1367636410,6b403925add4f628e7a1,2021-11-01,2021-11-30,1653.30
1,2232532355,02,0.00,10,2232532355,1811302322,6b403925add4f628e7a1,2022-12-01,2022-12-31,1573.60
2,2054823519,01,112.67,10,2054823519,1670163297,6b403925add4f628e7a1,2022-08-01,2022-08-31,1247.26
3,2224477516,01,0.00,10,2224477516,1804970927,a6978c24913ecd4b6447,2022-11-28,2022-11-30,-5.34
4,2300615680,01,104.00,10,2300615680,1863604510,6b403925add4f628e7a1,2023-02-01,2023-02-28,1172.08
...,...,...,...,...,...,...,...,...,...,...
201721,2016652502,01,0.00,10,2016652502,1638913314,66a004419bcdcae51c9a,2022-06-29,2022-06-30,-0.89
201722,2139981681,01,0.00,10,2139981681,1737987256,cbb1b7db3df747eacd09,2022-09-26,2022-09-30,-45.24
201723,2018513652,01,0.00,10,2018513652,1640272834,af0fe4ee5e8f5a3d1586,2022-06-20,2022-06-26,0.00
201724,2312099369,02,73.67,10,2312099369,1873795509,6b403925add4f628e7a1,2023-02-01,2023-02-28,0.00


In [22]:
# on recupère les id_employeur et id_assure
list_empass = list(versement['id_employeur_assure'])
empass = df_from_query(
    "SELECT DISTINCT id as id_employeur_assure, id_assure, id_employeur FROM dadeh.ddadtemployeur_assure WHERE "
    f"id IN ({list_to_enum(list_empass)}) "
    ";"
)
empass

,id_employeur_assure,id_assure,id_employeur
0,85169918,37337076,1017313
1,31259068,16582256,1347742
2,87862501,16299366,1250716
3,109282385,18535702,1117760
4,90626103,29753104,1916477
...,...,...,...
40601,70201820,6243940,1248833
40602,87000575,37692744,1256474
40603,110471251,7284067,1347742
40604,103119333,23363593,260010


In [61]:
# on a toutes les infos qu'il nous faut : les contrats IAE uniquement, les id_employeur et id_assure, les mesures en heures, les dates de paies
merged_with_assure = pd.merge(
    merged,
    pd.merge(
        empass,
        versement[['id', 'id_employeur_assure']].rename({'id': 'id_versement'}, axis=1),
        on='id_employeur_assure',
        how='left'
    ),
    on='id_versement',
    how='left',
)
merged_with_assure[['id_assure', 'id_employeur', 'date_debut_paie', 'date_fin_paie', 'mesure']].rename(
    {'id_assure': 'id_salarie', 'mesure': 'nb_heures_activite'},
    axis=1
)

,id_salarie,id_employeur,date_debut_paie,date_fin_paie,nb_heures_activite
0,36790573,840216,2021-11-01,2021-11-30,0.00
1,36615800,3755766,2022-12-01,2022-12-31,0.00
2,23291325,407209,2022-08-01,2022-08-31,112.67
3,30242310,4881063,2022-11-28,2022-11-30,0.00
4,26723926,2457832,2023-02-01,2023-02-28,104.00
...,...,...,...,...,...
201721,44297676,4025011,2022-06-29,2022-06-30,0.00
201722,17106332,4025011,2022-09-26,2022-09-30,0.00
201723,35664774,3393985,2022-06-20,2022-06-26,0.00
201724,37471376,5911082,2023-02-01,2023-02-28,73.67


In [25]:
# heures travaillées par assuré sur la période
hours_worked = merged_with_assure.groupby('id_assure')['mesure'].sum().reset_index()
hours_worked

,id_assure,mesure
0,14324,0.00
1,53434,770.82
2,89869,675.50
3,137725,3713.94
4,160442,566.81
...,...,...
11238,46310687,14.00
11239,46311834,14.00
11240,46311835,14.00
11241,46318890,84.00


In [90]:
# pour sélectionner un cas intéressant
merged_with_assure['id_assure'].value_counts().reset_index().iloc[70]
# merged_with_assure.drop_duplicates(['id_assure', 'id_employeur']).groupby('id_assure')['id_employeur'].count().sort_values()

id_assure    14308883
count             120
Name: 70, dtype: int64

In [93]:
# pour un assuré, toutes ses dates de paies et les heures travaillées
usecase = merged_with_assure.loc[
    (merged_with_assure['id_assure']==14308883)
#     & (merged_with_assure['mesure']>0)
].sort_values('date_debut_paie')
usecase = pd.merge(
    usecase[['date_debut_paie', 'id_employeur']].drop_duplicates(),
    usecase.groupby(['date_debut_paie', 'date_fin_paie'])['mesure'].sum().reset_index(),
    on='date_debut_paie',
    how='right',
)
usecase

,date_debut_paie,id_employeur,date_fin_paie,mesure
0,2021-02-11,1320323,2021-02-26,84.0
1,2021-03-01,1320323,2021-03-12,70.0
2,2021-07-13,1320323,2021-07-13,7.0
3,2021-07-15,1320323,2021-07-15,7.0
4,2021-07-16,1320323,2021-07-16,5.5
5,2021-07-21,1320323,2021-07-30,62.0
6,2021-08-01,1320323,2021-08-06,39.0
7,2021-08-09,1320323,2021-08-31,140.5
8,2021-09-01,1320323,2021-09-03,25.0
9,2021-09-06,1320323,2021-09-16,63.0


In [94]:
# nombre d'employeurs qu'a eus l'assuré
usecase['id_employeur'].value_counts()

id_employeur
1320323    25
5884455     8
Name: count, dtype: int64

In [95]:
# génération d'un document intelligible
md = ""
current_employeur = None
for _, row in usecase.iterrows():
    if row['id_employeur'] != current_employeur:
        current_employeur = row['id_employeur']
        md += f"## Employeur : {current_employeur}\n"
    md += f"- Du {row['date_debut_paie'].strftime('%d/%m/%Y')} au {row['date_fin_paie'].strftime('%d/%m/%Y')} : {row['mesure']} heures\n"
display(Markdown(md))

## Employeur : 1320323
- Du 11/02/2021 au 26/02/2021 : 84.0 heures
- Du 01/03/2021 au 12/03/2021 : 70.0 heures
- Du 13/07/2021 au 13/07/2021 : 7.0 heures
- Du 15/07/2021 au 15/07/2021 : 7.0 heures
- Du 16/07/2021 au 16/07/2021 : 5.5 heures
- Du 21/07/2021 au 30/07/2021 : 62.0 heures
- Du 01/08/2021 au 06/08/2021 : 39.0 heures
- Du 09/08/2021 au 31/08/2021 : 140.5 heures
- Du 01/09/2021 au 03/09/2021 : 25.0 heures
- Du 06/09/2021 au 16/09/2021 : 63.0 heures
- Du 24/06/2022 au 30/06/2022 : 35.0 heures
- Du 01/07/2022 au 15/07/2022 : 79.0 heures
- Du 18/07/2022 au 29/07/2022 : 75.0 heures
- Du 01/08/2022 au 05/08/2022 : 40.0 heures
- Du 08/08/2022 au 26/08/2022 : 114.5 heures
- Du 29/08/2022 au 31/08/2022 : 22.5 heures
- Du 01/09/2022 au 16/09/2022 : 90.5 heures
- Du 19/09/2022 au 30/09/2022 : 75.0 heures
- Du 01/10/2022 au 14/10/2022 : 67.5 heures
- Du 17/10/2022 au 28/10/2022 : 75.0 heures
- Du 31/10/2022 au 31/10/2022 : 7.5 heures
- Du 01/11/2022 au 18/11/2022 : 104.0 heures
- Du 21/11/2022 au 30/11/2022 : 64.0 heures
- Du 01/12/2022 au 02/12/2022 : 16.0 heures
- Du 05/12/2022 au 31/12/2022 : 105.0 heures
## Employeur : 5884455
- Du 01/01/2023 au 31/01/2023 : 194.0 heures
- Du 01/02/2023 au 28/02/2023 : 140.0 heures
- Du 14/03/2023 au 14/03/2023 : 7.0 heures
- Du 27/03/2023 au 27/03/2023 : 4.5 heures
- Du 03/04/2023 au 21/04/2023 : 112.0 heures
- Du 25/04/2023 au 30/04/2023 : 34.0 heures
- Du 01/05/2023 au 26/05/2023 : 161.5 heures
- Du 30/05/2023 au 31/05/2023 : 8.0 heures


In [54]:
df_cal = []
for _, row in usecase.iterrows():
    date_range = pd.date_range(row['date_debut_paie'], row['date_fin_paie'])
    df_cal.append(pd.DataFrame(
        {
            'date': date_range,
            'hours_on_period': [row['mesure'] for k in range(len(date_range))]
        }
    ))
df_cal = pd.concat(df_cal, ignore_index=True)
df_cal

,date,hours_on_period
0,2021-01-11,115.0
1,2021-01-12,115.0
2,2021-01-13,115.0
3,2021-01-14,115.0
4,2021-01-15,115.0
...,...,...
656,2023-05-27,85.0
657,2023-05-28,85.0
658,2023-05-29,85.0
659,2023-05-30,85.0


In [97]:
calplot(
    df_cal,
    x='date',
    y='hours_on_period',
#     color='id_employeur',
    years_title=True,
)

In [128]:
employeurs = list(contrat['id_employeur_assure'])
df_from_query(f"SELECT * FROM dadeh.ddadtemployeur WHERE id IN ({list_to_enum(employeurs)}) LIMIT 10")

,id,siret,et_enseigne,et_code_apet,et_ad_num_et_libelle_voie,et_ad_code_postal,et_ad_localite,et_ad_code_pays,et_ad_code_distrib_etranger,et_ad_complement_localisation,et_ad_service_distrib,et_ad_code_insee_commune,et_effectif_fin_periode,et_remuneration_expatries,et_nature_juridique,et_date_cloture,et_categorie_juridique,siren,nic_siege,en_code_apen,en_raison_sociale,en_ad_num_et_libelle_voie,en_ad_code_postal,en_ad_localite,en_ad_code_pays,en_ad_code_distrib_etranger,en_ad_complement_localisation,en_ad_service_distrib,en_effectif_moyen,en_code_implantation,en_date_debut_cvae,en_date_fin_cvae,mois_declaration,et_code_conv_collec_principale,et_operateur_competence,en_code_conv_collec_applicable
0,457540,BA7F68D4C7779F2B721D9A728D406214,enseigne,4619B,None,484,a,0,0,complement_localisation,service_distrib,0,None,None,None,None,1000,81065A8EC37DDB7D2096D5A5BECE50A5,F61A3D59DA608D990659F7F078240CE1,4619B,raison_sociale,None,484,a,0,0,complement_localisation,service_distrib,1,0,None,None,2022-03-01,9999,08,9999


In [118]:
df_from_query("SELECT * FROM dadeh.ddadtactivite LIMIT 10")

,id,id_remuneration_brute_ac,type,mesure,unite_mesure
0,2948747895,1839791032,01,-31.0,40
1,2949420650,1840099800,01,140.0,10
2,2949420651,1840099800,01,31.0,40
3,2948767156,1839799743,01,169.0,None
4,2948767157,1839799743,01,31.0,40
5,2948802452,1839814974,01,37.0,10
6,2948802453,1839814974,01,29.0,40
7,2948941378,1839879047,01,31.0,40
8,2951395915,1841024178,01,169.0,None
9,2951395916,1841024178,01,31.0,40
